# MXNet Model Server

This Notebook is borrowed from: https://github.com/TalkAI/facial-emotion-recognition-gluon/blob/master/notebooks/Gluon_FERPlus.ipynb

## Install the required Packages locally

We will need the PyPi packages listed below to test model server locally, and to perform image pre-processing prior to the model inference.

In [1]:
!pip install -q mxnet-model-server==1.0
print('mxnet-model-server installed')
!pip install -q scikit-image==0.13.0
print('scikit-learn installed')
!pip install -q opencv-python
print('opencv-python installed')

You are using pip version 10.0.1, however version 19.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
mxnet-model-server installed
You are using pip version 10.0.1, however version 19.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
scikit-learn installed
You are using pip version 10.0.1, however version 19.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
opencv-python installed


In [2]:
# We define the model's input and output type and shape via signature.json
# %cd /Users/kannanva/Documents/gbdc-mxnet-workshop/lab
%cat ./model_archive_resources/signature.json

{
  "input_type": "image/jpeg", 
  "inputs": [
    {
      "data_shape": [1, 1, 64, 64], 
      "data_name": "data"
    }
  ], 
  "outputs": [
    {
      "data_shape": [1, 8],
      "data_name": "hybridsequential0_dense2_fwd"
    }
  ], 
  "output_type": "application/json"
}

In [3]:
# We define the model's class label names via synset.txt
%cat ./model_archive_resources/synset.txt

neutral
happiness
surprise
sadness
anger
disgust
fear
contempt

In [4]:
# And lastly, we define custom code for request handling via python code other auxiliary files
%cat ./model_archive_resources/fer_service.py

# Copyright 2018 Amazon.com, Inc. or its affiliates. All Rights Reserved.
# Licensed under the Apache License, Version 2.0 (the "License").
# You may not use this file except in compliance with the License.
# A copy of the License is located at
#     http://www.apache.org/licenses/LICENSE-2.0
# or in the "license" file accompanying this file. This file is distributed
# on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either
# express or implied. See the License for the specific language governing
# permissions and limitations under the License.

import numpy as np
from mxnet_model_service import MXNetModelService
from mxnet_utils import image, ndarray
from skimage import transform
import mxnet as mx
import cv2 as cv
import logging

# One time initialization of Haar Cascade Classifier to extract and crop out face
face_detector = cv.CascadeClassifier('haarcascade_frontalface.xml')
# Classifier parameter specifying how much the image size is reduced at each image scale
s

## Download Model

In [5]:
!mkdir -p ~/SageMaker/ferplus
%pushd ~/SageMaker/ferplus
!curl -O https://s3.amazonaws.com/mxnet-demo-models/models/fer/gluon_ferplus-0000.params
!curl -O https://s3.amazonaws.com/mxnet-demo-models/models/fer/gluon_ferplus-symbol.json

!mv ~/SageMaker/ferplus/gluon_ferplus-0000.params ~/SageMaker/ferplus/ferplus-0000.params
!mv ~/SageMaker/ferplus/gluon_ferplus-symbol.json ~/SageMaker/ferplus/ferplus-symbol.json

/home/ec2-user/SageMaker/ferplus
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 33.4M  100 33.4M    0     0  12.7M      0  0:00:02  0:00:02 --:--:-- 12.7M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 16867  100 16867    0     0  44859      0 --:--:-- --:--:-- --:--:-- 44859


## Export model

In [6]:
%popd
%cp -r ./model_archive_resources/* ~/SageMaker/ferplus/

/home/ec2-user/SageMaker
popd -> ~/SageMaker


In [7]:
%pushd ~/SageMaker/ferplus

/home/ec2-user/SageMaker/ferplus


['~/SageMaker']

In [8]:
ls -al ./fer_service.py

-rw-rw-r-- 1 ec2-user ec2-user 3956 Jan 24 07:03 ./fer_service.py


In [9]:
# Let's package everything up into a Model Archive bundle
!model-archiver --model-name ferplus --handler fer_service:handle --model-path /home/ec2-user/SageMaker/ferplus --export-path /home/ec2-user/SageMaker/

ERROR - %s already exists.
Please specify --force/-f option to overwrite the model archive output file.
See -h/--help for more details./home/ec2-user/SageMaker/ferplus.mar


In [10]:
!ls -l ferplus.mar

ls: cannot access ferplus.mar: No such file or directory


In [11]:
# Spawning a new process to run the server
import subprocess as sp
server = sp.Popen("mxnet-model-server --start --model-store /home/ec2-user/SageMaker/ --models ferplus=ferplus.mar", shell=True)

In [12]:
server.pid

17090

In [13]:
# Check out the health endpoint
!curl http://localhost:8080/ping

{
  "status": "Healthy"
}


In [14]:
!ls

ferplus-0000.params    haarcascade_frontalface.xml  mxnet_utils
ferplus-symbol.json    logs			    signature.json
fer_service.py	       model_handler.py		    synset.txt
gluon_base_service.py  mxnet_model_service.py


In [15]:
%popd
test_image='happy.jpg'

/home/ec2-user/SageMaker
popd -> ~/SageMaker


In [16]:
from matplotlib.pyplot import imshow
from PIL import Image
import numpy as np
img = Image.open(test_image)
imshow(np.asarray(img))

In [17]:
!curl -X POST http://localhost:8080/predictions/ferplus -F "data=@$test_image"

[
  {
    "neutral": 0.3884313404560089
  },
  {
    "happiness": 0.5333355069160461
  },
  {
    "surprise": 0.002492635976523161
  },
  {
    "sadness": 0.05850818008184433
  },
  {
    "anger": 0.00885448046028614
  },
  {
    "disgust": 0.0009527787915430963
  },
  {
    "fear": 0.00076449871994555
  },
  {
    "contempt": 0.006660618353635073
  }
]

In [18]:
# Lastly, we'll terminate the server
server.terminate()